In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

case = pd.read_csv("Data Resources/Case.csv")
patient = pd.read_csv("Data Resources/PatientInfo.csv")
policy = pd.read_csv("Data Resources/Policy.csv")
region = pd.read_csv("Data Resources/Region.csv")
search = pd.read_csv("Data Resources/SearchTrend.csv")
seoul = pd.read_csv("Data Resources/SeoulFloating.csv")
time = pd.read_csv("Data Resources/Time.csv")
timeAge = pd.read_csv("Data Resources/TimeAge.csv")
timeGender = pd.read_csv("Data Resources/TimeGender.csv")
timeProvince = pd.read_csv("Data Resources/TimeProvince.csv")
weather = pd.read_csv("Data Resources/Weather.csv")

# Merging  AGE csv files sets to a Single Dataset

In [2]:
agePivot = timeAge.pivot(index='date', columns='age', values=['confirmed', 'deceased'])
genderPivot = timeAge.pivot(index='date', columns='age', values=['confirmed', 'deceased'])
provincePoint = timeAge.pivot(index='date', columns='age', values=['confirmed', 'deceased'])

In [3]:
agePivot.columns = ['-'.join(col) for col in agePivot.columns]
genderPivot.columns = ['-'.join(col) for col in genderPivot.columns]
provincePoint.columns = ['-'.join(col) for col in provincePoint.columns]

# Merging flattened datasets using df.concat()

In [4]:
pivots = pd.concat([agePivot, genderPivot, provincePoint], axis=1)

timeDF = time.merge(pivots, on='date', how='left')

# Prepare master Dataset

In [5]:
case = case.replace('-', np.nan)  # Replace dashes with NaN
df = patient.merge(case, on=['infection_case', 'province']
                   ,suffixes = (None, '_caseDF'), how='left', validate='many_to_one')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5165 entries, 0 to 5164
Data columns (total 20 columns):
patient_id            5165 non-null int64
sex                   4043 non-null object
age                   3785 non-null object
country               5165 non-null object
province              5165 non-null object
city                  5071 non-null object
infection_case        4246 non-null object
infected_by           1346 non-null object
contact_number        791 non-null object
symptom_onset_date    690 non-null object
confirmed_date        5162 non-null object
released_date         1587 non-null object
deceased_date         66 non-null object
state                 5165 non-null object
 case_id              4233 non-null float64
city_caseDF           1083 non-null object
group                 4233 non-null object
confirmed             4233 non-null float64
latitude              814 non-null object
longitude             814 non-null object
dtypes: float64(2), int64(1), object(1

In [6]:
# Dropping reduntant column and fix empty space in column name 

df = df.drop('city_caseDF', axis=1)
df.rename(columns={' case_id': 'case_id'}, inplace=True)

In [7]:
# Check whether region data is unique by province and city


df = df.loc[:,~df.columns.duplicated()]
df['province'].value_counts()
df['city'].value_counts()

Gyeongsan-si      639
Seongnam-si       173
Bucheon-si        162
Gwanak-gu         113
etc               110
                 ... 
sankyeock-dong      1
Kyeongsan-si        1
Taebaek-si          1
Ganghwa-gun         1
Gapyeong-gun        1
Name: city, Length: 163, dtype: int64

In [8]:
df = df.merge(region, on=['province', 'city'], how='left',
             suffixes=(None, '_regionDF'), validate='many_to_one')

# Adding a timeDF 

In [9]:
timeDF['join_date'] = pd.to_datetime(timeDF['date'], format='%Y-%m-%d')

In [10]:
df = df.merge(timeDF, left_on='confirmed_date', right_on='join_date',
              how='left', suffixes=(None, '_timeDF'), validate='many_to_one')


ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat